In [1]:
import pandas as pd

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'big_5_leagues_odds_5-11.csv'
big_df = pd.read_csv(file_path)
# Rename columns in big_df
big_df = big_df.rename(columns={
    'Combined Projected Home Goals': 'Projected Home Goals',
    'Combined Projected Away Goals': 'Projected Away Goals',
    'Combined Total Projected Goals': 'Projected Total Goals'
})

# Display the first few rows to verify the changes
print(big_df.head())


# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(big_df.head())


                Game Matchup  Projected Home Goals  Projected Away Goals  \
0          Alaves vs. Girona                  1.14                  0.96   
1  Athletic Club vs. Osasuna                  1.26                  0.73   
2    Granada vs. Real Madrid                  0.86                  1.28   
3    Mallorca vs. Las Palmas                  1.00                  0.63   
4     Villarreal vs. Sevilla                  1.36                  1.04   

   Projected Total Goals Home Win Probability Away Win Probability  \
0                   2.10               38.96%               31.63%   
1                   1.99               48.94%               20.92%   
2                   2.14               25.20%               47.06%   
3                   1.63               42.74%               22.97%   
4                   2.41               44.80%               28.31%   

  Draw Probability  Implied Home Odds  Implied Away Odds  Implied Draw Odds  \
0           29.41%                156      

In [2]:
import pandas as pd

# Assuming 'big_df' is your DataFrame and it has been defined earlier in your script.

# Function to apply bold text styling
def bold_text(value):
    return "font-weight: bold"

# Function to generate color backgrounds based on column type
def generate_color_background(column_type):
    def color_background(val):
        try:
            if isinstance(val, str) and "%" in val:
                num_val = float(val.rstrip('%'))
            else:
                num_val = float(val)
            
            if column_type == "probability":
                if num_val >= 50:
                    return 'background-color: lightgreen'
                elif num_val > 25 and num_val < 50:
                    return 'background-color: yellow'
                elif num_val <= 25:
                    return 'background-color: salmon'
            elif column_type == "edge":
                if num_val > 0.15:
                    return 'background-color: lightgreen'
                elif num_val <= 0.15:
                    return 'background-color: salmon'
        except ValueError:
            pass
        return ''
    return color_background

# Function to color goals based on comparison
def color_goals(row):
    colors = [''] * len(row)
    home_goals = float(row['Projected Home Goals'])
    away_goals = float(row['Projected Away Goals'])
    if home_goals > away_goals:
        colors[row.index.get_loc('Projected Home Goals')] = 'background-color: lightgreen'
        colors[row.index.get_loc('Projected Away Goals')] = 'background-color: salmon'
    elif away_goals > home_goals:
        colors[row.index.get_loc('Projected Home Goals')] = 'background-color: lightgreen'
        colors[row.index.get_loc('Projected Away Goals')] = 'background-color: salmon'
    return colors

# Odds formatter to remove unnecessary decimal places
def odds_formatter(value):
    try:
        float_val = float(value)
        if float_val.is_integer():
            return f"{int(float_val)}"
        else:
            return f"{float_val}"
    except ValueError:
        return value

# Initialize styled_df from big_df using the .style accessor
styled_df = big_df.style

# Apply all defined stylings and formatting
styled_df = styled_df.map(bold_text, subset=['Game Matchup'])\
    .applymap(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
    .applymap(generate_color_background("probability"), subset=['Home Win Probability', 'Away Win Probability', 'Draw Probability'])\
    .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
    .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
    .format({col: '{:.2%}' for col in ['Home Win Probability', 'Away Win Probability', 'Draw Probability']})\
    .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

# Note: This styled_df is ready for display in a Jupyter notebook or for exporting to HTML.
# To save the styled DataFrame as an HTML file: styled_df.to_html('styled_dataframe.html')


C:\Users\jbuck\AppData\Local\Temp\ipykernel_20548\1980033974.py:64: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
C:\Users\jbuck\AppData\Local\Temp\ipykernel_20548\1980033974.py:65: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(generate_color_background("probability"), subset=['Home Win Probability', 'Away Win Probability', 'Draw Probability'])\


In [3]:
styled_df

,Game Matchup,Projected Home Goals,Projected Away Goals,Projected Total Goals,Home Win Probability,Away Win Probability,Draw Probability,Implied Home Odds,Implied Away Odds,Implied Draw Odds,Home Odds,Away Odds,Draw Odds,Home Win Edge,Away Win Edge,Draw Edge,League
0,Alaves vs. Girona,1.14,0.96,2.10,38.96%,31.63%,29.41%,156,216,240,270,100,254,0.44,-0.37,0.04,La Liga
1,Athletic Club vs. Osasuna,1.26,0.73,1.99,48.94%,20.92%,30.14%,104,378,231,-200,525,330,-0.27,0.31,0.30,La Liga
2,Granada vs. Real Madrid,0.86,1.28,2.14,25.20%,47.06%,27.74%,297,112,260,459,-196,359,0.41,-0.29,0.28,La Liga
3,Mallorca vs. Las Palmas,1.00,0.63,1.63,42.74%,22.97%,34.29%,134,334,192,-153,500,260,-0.29,0.38,0.23,La Liga
4,Villarreal vs. Sevilla,1.36,1.04,2.41,44.80%,28.31%,26.89%,123,252,272,-120,280,309,-0.18,0.08,0.10,La Liga
5,Atletico Madrid vs. Celta Vigo,3.04,1.06,4.10,77.17%,9.86%,12.97%,-333,914,671,-222,500,390,0.12,-0.41,-0.36,La Liga
6,Cadiz vs. Getafe,1.09,0.53,1.62,49.27%,18.02%,32.71%,102,455,206,110,275,220,0.03,-0.32,0.05,La Liga
7,Real Betis vs. Almeria,1.16,0.74,1.90,46.45%,23.54%,30.01%,114,325,233,-277,650,440,-0.37,0.77,0.62,La Liga
8,Valencia vs. Rayo Vallecano,0.98,0.60,1.58,43.57%,21.27%,35.16%,129,370,184,125,245,215,-0.02,-0.27,0.11,La Liga
9,Barcelona vs. Real Sociedad,1.70,1.03,2.73,52.70%,23.45%,23.85%,-111,326,319,-125,330,275,-0.05,0.01,-0.11,La Liga


In [4]:
# Assuming styled_df is your styled DataFrame
html_file_path = '5-11_full_table.html'

# Save the styled DataFrame to an HTML file
styled_df.to_html(html_file_path)

print(f'The styled table has been saved to {html_file_path}')


The styled table has been saved to 5-11_full_table.html


In [5]:
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Full Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The table has been saved as an image to {image_file_path}')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Full Projections Table.png


In [6]:
import pandas as pd

# Assuming 'big_df' is your DataFrame and it has been defined earlier in your script.

# Filter the DataFrame for rows with any betting edge greater than 0.15
filtered_df = big_df[(big_df['Home Win Edge'] > 0.15) | (big_df['Away Win Edge'] > 0.15) | (big_df['Draw Edge'] > 0.15)]

# Additionally filter out edge columns that do not exceed the 0.15 threshold
edge_columns = ['Home Win Edge', 'Away Win Edge', 'Draw Edge']
for col in edge_columns:
    filtered_df.loc[filtered_df[col] <= 0.15, col] = None  # Set to NaN for columns under the threshold

# Function to apply bold text styling
def bold_text(value):
    return "font-weight: bold"

# Function to generate color backgrounds based on column type
def generate_color_background(column_type):
    def color_background(val):
        if pd.isna(val):
            return ''  # No styling for NaN values
        try:
            num_val = float(val)
            if column_type == "probability":
                if num_val >= 50:
                    return 'background-color: lightgreen'
                elif num_val > 25 and num_val < 50:
                    return 'background-color: yellow'
                elif num_val <= 25:
                    return 'background-color: salmon'
            elif column_type == "edge":
                if num_val > 0.15:
                    return 'background-color: lightgreen'
                elif num_val <= 0.15:
                    return 'background-color: salmon'
        except ValueError:
            pass
        return ''
    return color_background

# Function to color goals based on comparison
def color_goals(row):
    colors = [''] * len(row)
    home_goals = float(row['Projected Home Goals'])
    away_goals = float(row['Projected Away Goals'])
    if home_goals > away_goals:
        colors[row.index.get_loc('Projected Home Goals')] = 'background-color: lightgreen'
        colors[row.index.get_loc('Projected Away Goals')] = 'background-color: salmon'
    elif away_goals > home_goals:
        colors[row.index.get_loc('Projected Home Goals')] = 'background-color: lightgreen'
        colors[row.index.get_loc('Projected Away Goals')] = 'background-color: salmon'
    return colors

# Odds formatter to remove unnecessary decimal places
def odds_formatter(value):
    try:
        float_val = float(value)
        if float_val.is_integer():
            return f"{int(float_val)}"
        else:
            return f"{float_val}"
    except ValueError:
        return value

# Initialize styled_df from filtered_df using the .style accessor
only_bets_styled_df = filtered_df.style

# Apply all defined stylings and formatting
only_bets_styled_df = only_bets_styled_df.applymap(bold_text, subset=['Game Matchup'])\
    .applymap(generate_color_background("edge"), subset=edge_columns)\
    .applymap(generate_color_background("probability"), subset=['Home Win Probability', 'Away Win Probability', 'Draw Probability'])\
    .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
    .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
    .format({col: '{:.2%}' for col in ['Home Win Probability', 'Away Win Probability', 'Draw Probability']})\
    .format({col: '{:.2f}' for col in edge_columns + ['Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

# Note: This styled_df is ready for display in a Jupyter notebook or for exporting to HTML.
# To save the styled DataFrame as an HTML file: styled_df.to_html('styled_dataframe.html')


C:\Users\jbuck\AppData\Local\Temp\ipykernel_20548\1670822874.py:69: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  only_bets_styled_df = only_bets_styled_df.applymap(bold_text, subset=['Game Matchup'])\
C:\Users\jbuck\AppData\Local\Temp\ipykernel_20548\1670822874.py:70: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(generate_color_background("edge"), subset=edge_columns)\
C:\Users\jbuck\AppData\Local\Temp\ipykernel_20548\1670822874.py:71: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(generate_color_background("probability"), subset=['Home Win Probability', 'Away Win Probability', 'Draw Probability'])\


In [7]:
only_bets_styled_df

,Game Matchup,Projected Home Goals,Projected Away Goals,Projected Total Goals,Home Win Probability,Away Win Probability,Draw Probability,Implied Home Odds,Implied Away Odds,Implied Draw Odds,Home Odds,Away Odds,Draw Odds,Home Win Edge,Away Win Edge,Draw Edge,League
0,Alaves vs. Girona,1.14,0.96,2.10,38.96%,31.63%,29.41%,156,216,240,270,100,254,0.44,nan,nan,La Liga
1,Athletic Club vs. Osasuna,1.26,0.73,1.99,48.94%,20.92%,30.14%,104,378,231,-200,525,330,nan,0.31,0.30,La Liga
2,Granada vs. Real Madrid,0.86,1.28,2.14,25.20%,47.06%,27.74%,297,112,260,459,-196,359,0.41,nan,0.28,La Liga
3,Mallorca vs. Las Palmas,1.00,0.63,1.63,42.74%,22.97%,34.29%,134,334,192,-153,500,260,nan,0.38,0.23,La Liga
7,Real Betis vs. Almeria,1.16,0.74,1.90,46.45%,23.54%,30.01%,114,325,233,-277,650,440,nan,0.77,0.62,La Liga
13,Bournemouth vs. Brentford,1.78,1.63,3.42,41.97%,35.41%,22.62%,138,181,342,100,249,301,nan,0.24,nan,Premier League
15,Fulham vs. Manchester City,0.92,1.63,2.56,21.35%,54.32%,24.33%,368,-119,311,1184,-454,602,1.74,nan,0.71,Premier League
16,Newcastle United vs. Brighton,3.17,0.66,3.83,86.06%,4.77%,9.17%,-625,1996,991,-163,376,388,0.39,nan,nan,Premier League
17,Nottingham Forest vs. Chelsea,1.70,1.29,2.99,47.31%,29.06%,23.63%,110,244,323,249,102,291,0.65,nan,nan,Premier League
18,Tottenham vs. Burnley,2.41,1.51,3.93,57.34%,23.51%,19.15%,-135,325,422,-277,606,498,nan,0.66,nan,Premier League


In [8]:
import pandas as pd

# Function to convert American odds to decimal odds
def american_to_decimal(odds):
    if odds > 0:
        return (odds / 100) + 1
    elif odds < 0:
        return (100 / abs(odds)) + 1
    else:
        return 1  # Handle scenario where odds might be zero

# Function to apply the Kelly Criterion to calculate the bet amount
def kelly_criterion(row, base_max_bet):
    decimal_odds = american_to_decimal(row['Relevant Odds'])
    implied_decimal_odds = american_to_decimal(row['Relevant Implied Odds'])
    b = decimal_odds - 1
    p = 1 / implied_decimal_odds
    q = 1 - p
    kelly_fraction = (b * p - q) / b if b != 0 else 0
    kelly_bet = kelly_fraction * total_bankroll
    adjusted_max_bet = base_max_bet * (row['edge'] / 0.25)
    return kelly_fraction, min(max(0, kelly_bet), adjusted_max_bet)

# Reshape the 'edge' columns and filter
edges_melted = big_df.melt(
    id_vars=['Game Matchup', 'Projected Home Goals', 'Projected Away Goals', 'Home Odds', 'Away Odds', 'Draw Odds', 'Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds'],
    value_vars=['Home Win Edge', 'Away Win Edge', 'Draw Edge'],
    var_name='edge_type',
    value_name='edge'
).query('edge > 0.25')

edges_melted['edge_type'] = edges_melted['edge_type'].map({
    'Home Win Edge': 'Home Edge',
    'Away Win Edge': 'Away Edge',
    'Draw Edge': 'Draw Edge'
})

# Extract relevant odds and implied odds based on the type of edge
edges_melted[['Relevant Odds', 'Relevant Implied Odds']] = edges_melted.apply(
    lambda row: pd.Series([row[f'{row["edge_type"].split()[0]} Odds'], row[f'Implied {row["edge_type"].split()[0]} Odds']]),
    axis=1
)

# Define total bankroll and base maximum bet per game
total_bankroll = 1000
base_max_bet_per_game = 100

# Calculate bet amounts and kelly fraction using the Kelly Criterion
edges_melted[['Kelly Fraction', 'Bet Amount']] = edges_melted.apply(lambda row: pd.Series(kelly_criterion(row, base_max_bet_per_game)), axis=1)

# Normalize if the total bets exceed the total bankroll
total_bets = edges_melted['Bet Amount'].sum()
if total_bets > total_bankroll:
    normalization_factor = total_bankroll / total_bets
    edges_melted['Bet Amount'] *= normalization_factor

# Calculate 'To Win' based on the bet amount and the odds
edges_melted['Decimal Odds'] = edges_melted['Relevant Odds'].apply(american_to_decimal)
edges_melted['To Win'] = edges_melted['Bet Amount'] * (edges_melted['Decimal Odds'] - 1)

# Styling the DataFrame
kc_styled_df = edges_melted.style.apply(
    lambda x: ['font-weight: bold' if x.name == 'Game Matchup' else '' for _ in x],
    axis=1
).apply(
    lambda x: ['background-color: lightgreen' if v > 0.15 else 'background-color: salmon' for v in x],
    subset=['edge'],
    axis=1
).format({
    'Kelly Fraction': "{:.2%}",
    'Decimal Odds': "{:.2f}",
    'Decimal Implied Odds': "{:.2f}",
    'Implied Probability': "{:.2%}",
    'Bet Amount': "${:,.2f}",
    'To Win': "${:,.2f}"
})

# Display or export the final DataFrame as HTML
kc_styled_html = kc_styled_df.to_html()
print(kc_styled_html)  # This will print the HTML string which can be displayed in Jupyter or saved to an HTML file


<style type="text/css">
#T_fc979_row0_col10, #T_fc979_row1_col10, #T_fc979_row2_col10, #T_fc979_row3_col10, #T_fc979_row4_col10, #T_fc979_row5_col10, #T_fc979_row6_col10, #T_fc979_row7_col10, #T_fc979_row8_col10, #T_fc979_row9_col10, #T_fc979_row10_col10, #T_fc979_row11_col10, #T_fc979_row12_col10, #T_fc979_row13_col10, #T_fc979_row14_col10, #T_fc979_row15_col10, #T_fc979_row16_col10, #T_fc979_row17_col10, #T_fc979_row18_col10, #T_fc979_row19_col10, #T_fc979_row20_col10, #T_fc979_row21_col10, #T_fc979_row22_col10, #T_fc979_row23_col10, #T_fc979_row24_col10, #T_fc979_row25_col10, #T_fc979_row26_col10, #T_fc979_row27_col10, #T_fc979_row28_col10, #T_fc979_row29_col10, #T_fc979_row30_col10 {
  background-color: lightgreen;
}
</style>
<table id="T_fc979">
  <thead>
    <tr>
      <th class="blank level0" >&nbsp;</th>
      <th id="T_fc979_level0_col0" class="col_heading level0 col0" >Game Matchup</th>
      <th id="T_fc979_level0_col1" class="col_heading level0 col1" >Projected Home Goals</

In [9]:
kc_styled_df

,Game Matchup,Projected Home Goals,Projected Away Goals,Home Odds,Away Odds,Draw Odds,Implied Home Odds,Implied Away Odds,Implied Draw Odds,edge_type,edge,Relevant Odds,Relevant Implied Odds,Kelly Fraction,Bet Amount,Decimal Odds,To Win
0,Alaves vs. Girona,1.140000,0.960000,270,100,254,156,216,240,Home Edge,0.440000,270,156,16.49%,$34.95,3.70,$94.35
2,Granada vs. Real Madrid,0.860000,1.280000,459,-196,359,297,112,260,Home Edge,0.410000,459,297,8.89%,$18.84,5.59,$86.46
15,Fulham vs. Manchester City,0.920000,1.630000,1184,-454,602,368,-119,311,Home Edge,1.740000,1184,368,14.73%,$31.20,12.84,$369.44
16,Newcastle United vs. Brighton,3.170000,0.660000,-163,376,388,-625,1996,991,Home Edge,0.390000,-163,-625,63.72%,$33.05,1.61,$20.28
17,Nottingham Forest vs. Chelsea,1.700000,1.290000,249,102,291,110,244,323,Home Edge,0.650000,249,110,26.58%,$55.09,3.49,$137.17
23,Tottenham vs. Manchester City,1.400000,1.780000,499,-232,411,219,117,344,Home Edge,0.880000,499,219,17.59%,$37.27,5.99,$185.98
27,Atalanta vs. Roma,2.840000,1.630000,100,265,260,-169,401,480,Home Edge,0.260000,100,-169,25.65%,$22.04,2.00,$22.04
28,Genoa vs. Sassuolo,2.220000,0.690000,120,220,254,-270,922,488,Home Edge,0.610000,120,-270,50.45%,$51.70,2.20,$62.04
35,Borussia M.Gladbach vs. Eintracht Frankfurt,2.330000,1.310000,150,170,265,-151,390,413,Home Edge,0.500000,150,-151,33.60%,$42.38,2.50,$63.56
42,Darmstadt vs. Hoffenheim,1.730000,2.110000,350,-161,370,209,110,393,Home Edge,0.460000,350,209,13.04%,$27.62,4.50,$96.68


In [10]:
# Assuming styled_df is your styled DataFrame
html_file_path = '5-11_kc_table.html'

# Save the styled DataFrame to an HTML file
kc_styled_df.to_html(html_file_path)

print(f'The styled table has been saved to {html_file_path}')


The styled table has been saved to 5-11_kc_table.html


In [11]:
# Assuming styled_df is your styled DataFrame
html_file_path = '5-11_only_bets.html'

# Save the styled DataFrame to an HTML file
only_bets_styled_df.to_html(html_file_path)

print(f'The styled table has been saved to {html_file_path}')


The styled table has been saved to 5-11_only_bets.html


In [12]:
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = only_bets_styled_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Only Bets.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The table has been saved as an image to {image_file_path}')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Only Bets.png


In [13]:
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = kc_styled_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Kelly Criteron Model Projections For May 11th Weekend Games.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The table has been saved as an image to {image_file_path}')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Kelly Criteron Model Projections For May 11th Weekend Games.png


In [14]:
# Assuming styled_df is your styled DataFrame
html_file_path = '5-11_projections_table.html'

# Save the styled DataFrame to an HTML file
styled_df.to_html(html_file_path)

print(f'The styled table has been saved to {html_file_path}')


The styled table has been saved to 5-11_projections_table.html


In [15]:
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Model Projections For Weekend of May 11th Games.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The table has been saved as an image to {image_file_path}')


Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Model Projections For Weekend of May 11th Games.png


In [14]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
premier_league_df = big_df[big_df['League'].str.contains("Premier League", na=False)]

# Check if the filtered DataFrame is not empty
if not premier_league_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_premier_league_df = premier_league_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'premier_league_table.html'
    styled_premier_league_df.to_html(html_file_path)
    print(f"The Premier League table has been saved to {html_file_path}")
else:
    print("There are no Premier League teams in the table.")

import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_premier_league_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 3rd Weekend\EPL Projections Table.png'
imgkit.from_string(html_data, image_file_path, config=config)
print(f'The Premier League table has been saved as an image to {image_file_path}')



The Premier League table has been saved to premier_league_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The Premier League table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 3rd Weekend\EPL Projections Table.png


In [16]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
la_liga_df = big_df[big_df['League'].str.contains("La Liga", na=False)]

# Check if the filtered DataFrame is not empty
if not la_liga_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_la_liga_df = la_liga_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'la_liga_table.html'
    styled_la_liga_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no La Liga teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_la_liga_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\La Liga Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


The La Liga table has been saved to la_liga_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The La Liga table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\La Liga Projections Table.png


In [17]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
bundesliga_df = big_df[big_df['League'].str.contains("Bundesliga", na=False)]

# Check if the filtered DataFrame is not empty
if not bundesliga_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_bundesliga_df = bundesliga_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'bundesliga_table.html'
    styled_bundesliga_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no La Liga teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_bundesliga_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Bundesliga Projections Table.png'


imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


The La Liga table has been saved to bundesliga_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The La Liga table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Bundesliga Projections Table.png


In [18]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
ligue_une_df = big_df[big_df['League'].str.contains("Ligue Une", na=False)]

# Check if the filtered DataFrame is not empty
if not ligue_une_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_ligue_une_df = ligue_une_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'ligue_une_table.html'
    styled_ligue_une_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no Ligue Une teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_ligue_une_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Ligue Une Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


The La Liga table has been saved to ligue_une_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The La Liga table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Ligue Une Projections Table.png


In [19]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
serie_a_df = big_df[big_df['League'].str.contains("Serie A", na=False)]

# Check if the filtered DataFrame is not empty
if not serie_a_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_serie_a_df = serie_a_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'serie_a_table.html'
    styled_serie_a_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no Ligue Une teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_serie_a_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Serie A Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


The La Liga table has been saved to serie_a_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The La Liga table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 11th Weekend\Serie A Projections Table.png


In [19]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
eredivisie_df = big_df[big_df['League'].str.contains("Eredivisie", na=False)]

# Check if the filtered DataFrame is not empty
if not eredivisie_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_eredivisie_df = eredivisie_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'eredivisie_table.html'
    styled_eredivisie_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no Ligue Une teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_eredivisie_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 3rd Weekend\Eredivisie Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


There are no Ligue Une teams in the table.


NameError: name 'styled_eredivisie_df' is not defined

In [20]:
# Assuming the necessary functions like bold_text, generate_color_background, color_goals, and odds_formatter are already defined as in previous examples.

# Define the columns you want to format as percentages
percent_cols = ['Home Win Probability', 'Away Win Probability', 'Draw Probability']

# Continue with your previous code, now with percent_cols defined
efl_df = big_df[big_df['League'].str.contains("EFL", na=False)]

# Check if the filtered DataFrame is not empty
if not efl_df.empty:
    # Apply the styling to the filtered DataFrame
    styled_efl_df = efl_df.style\
        .map(bold_text, subset=['Game Matchup'])\
        .map(generate_color_background("edge"), subset=['Home Win Edge', 'Away Win Edge', 'Draw Edge'])\
        .map(generate_color_background("probability"), subset=percent_cols)\
        .apply(color_goals, axis=1, subset=['Projected Home Goals', 'Projected Away Goals'])\
        .set_table_styles([{'selector': 'th', 'props': [('font-style', 'italic')]}], overwrite=False)\
        .set_properties(**{'border': '1px solid black'})\
        .format({col: odds_formatter for col in ['Implied Home Odds', 'Implied Away Odds', 'Implied Draw Odds', 'Home Odds', 'Away Odds', 'Draw Odds']}, na_rep="-")\
        .format({col: '{:.2%}' for col in percent_cols})\
        .format({col: '{:.2f}' for col in ['Home Win Edge', 'Away Win Edge', 'Draw Edge', 'Projected Home Goals', 'Projected Away Goals', 'Projected Total Goals']})

    # Now you can display the styled DataFrame in a Jupyter notebook directly,
    # or save it to an HTML file if needed
    html_file_path = 'eredivisie_table.html'
    styled_efl_df.to_html(html_file_path)
    print(f"The La Liga table has been saved to {html_file_path}")
else:
    print("There are no Ligue Une teams in the table.")
import imgkit

# Specify the full path to the wkhtmltoimage executable
path_to_wkhtmltoimage = r'C:\Users\jbuck\wkhtmltopdf\bin\wkhtmltoimage.exe'  # Adjusted path

config = imgkit.config(wkhtmltoimage=path_to_wkhtmltoimage)

html_data = styled_efl_df.to_html()

# Save the HTML table as an image
image_file_path = r'C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 3rd Weekend\EFL Projections Table.png'

imgkit.from_string(html_data, image_file_path, config=config)
print(f'The La Liga table has been saved as an image to {image_file_path}')


The La Liga table has been saved to eredivisie_table.html
Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               
The La Liga table has been saved as an image to C:\Users\jbuck\Dropbox\Soccer_Projects\Betting\Model Projections\May 3rd Weekend\EFL Projections Table.png
